In [1]:
import model.nn.esn as esn
import Preprocessing.preprocessing as preprocessing
import torch
from torch.utils.data import DataLoader
from model.nn import esn
from model.utils.utils import prepare_target
import os

In [2]:
# WAV 파일이 있는 디렉토리
wav_dir_train = '/Users/imdohyeon/Documents/PythonWorkspace/Lieon-ai/dataset/train'
wav_files_train = [os.path.join(wav_dir_train, file) for file in os.listdir(wav_dir_train) if file.endswith('.wav')]

wav_dir_test = '/Users/imdohyeon/Documents/PythonWorkspace/Lieon-ai/dataset/test'
wav_files_test = [os.path.join(wav_dir_test, file) for file in os.listdir(wav_dir_test) if file.endswith('.wav')]

In [3]:
"""
데이터 로드 및 전처리, 데이터로더 생성
"""
# 데이터셋 생성
train_dataset = preprocessing.WAVDataset(wav_files_train)
test_dataset = preprocessing.WAVDataset(wav_files_test)

# 데이터로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [4]:
"""
모델 정의, 학습 및 예측
"""
# ESN 모델 파라미터 정의
input_size = 24  # feature 개수
hidden_size = 100
output_size = 1  # 출력 값이 0.5보다 작으면 클래스 0(거짓말 아님), 0.5보다 크면 클래스 1(거짓말)
num_layers = 1  # Reservoir 개수
leaking_rate = 0.2  # 값이 작을수록 이전 hidden 상태가 더 많이 유지
spectral_radius = 0.9
w_ih_scale = 1.0  # 첫 번째 레이어의 입력 가중치 스케일
density = 1.0  # 순환 가중치 행렬의 밀도 (1이면 모든 요소가 nonzero)
lambda_reg = 1e-4  # L2(Ridge) regularization 가중치(축소 파라미터) - 클 수록 강한 규제(과적합 방지)
readout_training = 'svd'  # Readout 학습 알고리즘 지정 (svd, cholesky, inv, gd)
output_steps = 'all'  # ridge regression 방법에서 reservoir 출력을 사용하는 방법 (last, all, mean)

# ESN 모델 생성
model = esn.ESN(input_size, hidden_size, output_size, num_layers=num_layers,
            nonlinearity='tanh', leaking_rate=leaking_rate, spectral_radius=spectral_radius,
            w_ih_scale=1.0, lambda_reg=lambda_reg, density=density, w_io=True, readout_training=readout_training, output_steps=output_steps)

In [7]:
# 모델 학습 예제
for batch_data, batch_washout, batch_targets in train_dataloader:
    # 초기 hidden state 설정 (필요에 따라 None으로 두어도 됨)
    h_0 = torch.zeros(esn.num_layers, batch_data.size(0), esn.hidden_size)

    # ESN 모델에 데이터 전달
    output, hidden = esn(batch_data, batch_washout, h_0, batch_targets)

    # 손실 함수 및 최적화 알고리즘 정의 (여기서는 예제로 MSE 손실 사용)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(esn.parameters())

    # 손실 계산 및 역전파
    if output is not None:
        loss = criterion(output, batch_targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # fit 함수 호출 (필요 시)
    esn.fit()

    print(f'Loss: {loss.item()}')


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 61, in fetch
    return self.collate_fn(data)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 143, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 143, in <listcomp>
    return [collate(sampl

## Test2

In [12]:
import torch
from torch.utils.data import DataLoader
from model.nn.esn import ESN
from Preprocessing.preprocessing2 import create_dataloader
import os

# Parameters
# WAV 파일이 있는 디렉토리
wav_dir_train = '/Users/imdohyeon/Documents/PythonWorkspace/Lieon-ai/dataset/train'
wav_files_train = [os.path.join(wav_dir_train, file) for file in os.listdir(wav_dir_train) if file.endswith('.wav')]

wav_dir_test = '/Users/imdohyeon/Documents/PythonWorkspace/Lieon-ai/dataset/test'
wav_files_test = [os.path.join(wav_dir_test, file) for file in os.listdir(wav_dir_test) if file.endswith('.wav')]


max_length = 200  # Set maximum length for padding/truncating
batch_size = 4
leaking_rate = 1.0
spectral_radius = 0.9
lambda_reg = 1e-4
washout = 10
input_size = 23  # feature 개수
hidden_size = 100
output_size = 1  # 출력 값이 0.5보다 작으면 클래스 0(거짓말 아님), 0.5보다 크면 클래스 1(거짓말)
num_layers = 1  # Reservoir 개수
w_ih_scale = 1.0  # 첫 번째 레이어의 입력 가중치 스케일
density = 1.0  # 순환 가중치 행렬의 밀도 (1이면 모든 요소가 nonzero)
readout_training = 'svd'  # Readout 학습 알고리즘 지정 (svd, cholesky, inv, gd)
output_steps = 'all'  # ridge regression 방법에서 reservoir 출력을 사용하는 방법 (last, all, mean)


In [13]:
# Create DataLoader
dataloader = create_dataloader(wav_files_train, max_length, batch_size, shuffle=True)

# Initialize ESN
esn = ESN(input_size=input_size, hidden_size=hidden_size, output_size=output_size,
          num_layers=num_layers, leaking_rate=leaking_rate, spectral_radius=spectral_radius,
          lambda_reg=lambda_reg, batch_first=True)

# Training loop
num_epochs = 10
criterion = torch.nn.MSELoss()  # Example loss function
optimizer = torch.optim.Adam(esn.parameters(), lr=1e-3)

In [21]:
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        # Zero gradients
        optimizer.zero_grad()
        
        # Prepare washout for each sample in the batch
        washout_tensor = torch.tensor([washout] * inputs.size(0))
        
        # Forward pass
        outputs, _ = esn(inputs, washout_tensor)
        
        # Remove the washout period from the outputs and targets
        outputs = outputs[:, washout:, :]
        targets = targets[:, washout:]
        
        min_length = min(outputs.size(1), targets.size(1))
        outputs = outputs[:, :min_length, :]  # Truncate outputs to min length
        targets = targets[:, :min_length]     # Truncate targets to min length
        
        outputs = outputs.squeeze(-1) 
        
        # Compute loss
        loss = criterion(outputs, targets)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/10, Loss: 4.252281188964844
Epoch 2/10, Loss: 3.964891195297241
Epoch 3/10, Loss: 3.689911127090454
Epoch 4/10, Loss: 3.427537679672241
Epoch 5/10, Loss: 3.177931308746338
Epoch 6/10, Loss: 2.941208600997925
Epoch 7/10, Loss: 2.717440366744995
Epoch 8/10, Loss: 2.506647825241089
Epoch 9/10, Loss: 2.308809518814087
Epoch 10/10, Loss: 2.1238598823547363


In [22]:
# Save the trained model
torch.save(esn.state_dict(), 'esn_model_test.pth')